In [ ]:
; cd '~/Dropbox/code/PowerSimulations.jl'

In [ ]:
] activate .; instantiate; build; st

In [ ]:
include("src/cvar.jl")

In [ ]:
include(joinpath("/Users/jdlara/Documents/UCB/NREL Work/get_ERCOT_data.jl"))

In [ ]:
branches = Dict{Symbol, DeviceModel}(:L => DeviceModel(PSY.Line, PSI.StaticLine),
                                     :T => DeviceModel(PSY.Transformer2W, PSI.StaticTransformer),
                                     :TT => DeviceModel(PSY.TapTransformer, PSI.StaticTransformer),
                                     :dc_line => DeviceModel(PSY.HVDCLine, PSI.HVDCDispatch))

services = Dict{Symbol, PSI.ServiceModel}()

devices = Dict{Symbol, DeviceModel}(:Generators => DeviceModel(PSY.ThermalStandard, PSI.ThermalUnitCommitment),
                                    :Ren => DeviceModel(PSY.RenewableDispatch, PSI.RenewableFullDispatch),
                                    :Loads =>  DeviceModel(PSY.PowerLoad, PSI.StaticPowerLoad))       

#Model_ref holds all the details of the model to be built 
model_ref= ModelReference(CopperPlatePowerModel, devices, branches, services);

In [ ]:
OpModel = OperationModel(TestOptModel, model_ref, sys2; optimizer = CPLEX_optimizer, parameters = false, forecast=false)

In [ ]:
OpModel.canonical_model.variables